# Classification plant phenotype

This is an example of a non-geospatial use of the MapReader pipeline

# Load

https://mapreader.readthedocs.io/en/latest/User-guide/Load.html

## Load images

In [ ]:
from mapreader import loader

my_files = loader("./dataset/open_access_plant/*.png")

In [ ]:
# len() shows the total number of images currently read (or sliced, see below)
print(f"Number of images: {len(my_files)}")

In [ ]:
print(my_files)

In [ ]:
my_files.show_sample(num_samples=2, tree_level="parent")

## Patchify images

In [ ]:
my_files.patchify_all(patch_size=50)  # in pixels

In [ ]:
my_files.show_sample(num_samples=6, tree_level="patch")

In [8]:
parent_list = my_files.list_parents()
patch_list = my_files.list_patches()

In [ ]:
my_files.show_patches(
    parent_list[0], figsize=(15, 15)
)

## Calculate pixel intensities

In [ ]:
my_files.calc_pixel_stats()

In [ ]:
parent_df, patch_df = my_files.convert_images(save=True)

In [ ]:
parent_df.head()

In [ ]:
patch_df.head()

In [ ]:
patch_df.mean()

In [ ]:
my_files.show_patches(
    parent_list[0],
    column_to_plot="mean_pixel_G",
    vmin=0.1,
    vmax=0.6,
    figsize=(20, 20),
    alpha=0.7,
)

# Annotate
https://mapreader.readthedocs.io/en/latest/User-guide/Annotate.html

## Set up inputs

In [20]:
from mapreader import Annotator

In [ ]:
annotator = Annotator(
    patch_paths="./patches_50_pixel/*png", 
    parent_paths="./dataset/open_access_plant/*png",
    metadata_path="./parent_df.csv",
    annotations_dir= "./annotations_phenotype_open_access",
    username = "kasra",
    task_name = "phenotype",
    labels =["No", "Plant"],
    sortby="mean_pixel_G",
    show_context=True,
)

In [ ]:
annotator.annotate(resize_to=500, sortby="mean_pixel_G", ascending=False)

# Classify - Train
https://mapreader.readthedocs.io/en/latest/User-guide/Classify.html

## Read in annotations

In [23]:
from mapreader import AnnotationsLoader

In [ ]:
annotated_images = AnnotationsLoader()

annotated_images.load("./annotations_phenotype_open_access/phenotype_test_#kasra#.csv", images_dir="./patches_50_pixel/")

In [ ]:
annotated_images.annotations

In [ ]:
print(annotated_images)

In [ ]:
annotated_images.annotations.index[5]

In [ ]:
# show sample images for one labe (label_to_show)
annotated_images.show_sample(label_to_show="Plant")

In [ ]:
# show an image based on its index
annotated_images.show_patch("patch-300-300-350-350-#2014-06-06_plant001_rgb.png#.png")

## Prepare datasets and dataloaders

In [ ]:
annotated_images.create_datasets(frac_train=0.7, frac_val=0.2, frac_test=0.1)

In [ ]:
annotated_images.labels_map

In [ ]:
dataloaders = annotated_images.create_dataloaders(batch_size=16)

## Set up `ClassifierContainer`

### Load a pretrained PyTorch model, dataloaders and labels map

In [33]:
from mapreader import ClassifierContainer

In [ ]:
my_classifier = ClassifierContainer(model ="resnet18",
                                    labels_map={0: 'No', 1: 'plant'},
                                    dataloaders=dataloaders
                                    )

In [ ]:
my_classifier.labels_map

In [ ]:
my_classifier.model_summary()

### Set up loss function, optimizer and scheduler

In [ ]:
my_classifier.add_loss_fn()

In [39]:
my_classifier.initialize_optimizer()

In [40]:
my_classifier.initialize_scheduler()

## Train/fine-tune a model

In [ ]:
my_classifier.train(
    num_epochs=10,
    tmp_file_save_freq=2,
    remove_after_load=False,
    print_info_batch_freq=5,
)

In [ ]:
my_classifier.list_metrics()

In [ ]:
my_classifier.plot_metric(
    metrics=["loss"],
)

In [ ]:
my_classifier.plot_metric(
    metrics=['precision_macro', 'recall_macro','fscore_macro','support_macro','rocauc_macro'],
)

# Classify - Infer

https://mapreader.readthedocs.io/en/latest/User-guide/Classify.html

## Create dataset with all patches and add to ``ClassifierContainer``

In [45]:
from mapreader import PatchDataset

In [ ]:
data = PatchDataset(patch_df, transform="val")

In [47]:
my_classifier.load_dataset(data, set_name="all_plants")

## Infer labels

In [ ]:
my_classifier.inference("all_plants")

In [ ]:
my_classifier.show_inference_sample_results(
    label="plant", set_name="all_plants", min_conf=99
)

## Save results

In [ ]:
my_classifier.save_predictions("all_plants")

## Add outputs to `MapImages` as metadata

In [ ]:
my_files.add_metadata("all_plants_predictions_patch_df.csv", tree_level="patch", ignore_mismatch=True)

In [ ]:
parent_df, patch_df = my_files.convert_images()
patch_df.head()

In [55]:
parent_list = my_files.list_parents()

In [ ]:
my_files.show_patches(
    parent_list[0],
    column_to_plot="pred",
    figsize=(15, 15),
    alpha=0.5,
)

In [ ]:
my_files.show_patches(
    parent_list[0],
    column_to_plot="conf",
    figsize=(15, 15),
    alpha=0.5,
)